<a href="https://colab.research.google.com/github/WinterBlue16/MultcampusAI_FinalProject/blob/master/Multiple_classification(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!unzip -uq "/content/drive/My Drive/roi_sample/roi_set.zip" -d "/content/drive/My Drive/roi_sample"

In [0]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np

import cv2
import os
import glob

In [0]:
path_plus='/content/drive/My Drive/roi_sample/e01'
path_zero='/content/drive/My Drive/roi_sample/e02'
path_minus='/content/drive/My Drive/roi_sample/e03'
# path_unkno='image/unknown'

la=len(os.listdir(path_zero))
lb=len(os.listdir(path_plus))
lc=len(os.listdir(path_minus))
# lc=len(os.listdir(path_unkno)) 

print('zero 경로에 저장된 파일의 개수:'+str(la))
print('plue 경로에 저장된 파일의 개수:'+str(lb))
print('minus 경로에 저장된 파일의 개수:'+str(lc))

plus 경로에 저장된 파일의 개수:500
zero 경로에 저장된 파일의 개수:500
minus 경로에 저장된 파일의 개수:500


In [0]:
xsize=200
ysize=200

In [0]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [0]:
read_dir(path_plus, 0) #'plus emotion')
read_dir(path_zero, 1) #'zero emotion')
read_dir(path_minus, 2) #'minus emotion')

In [0]:
x=np.array(x)
y=np.array(y)

In [0]:
print(x.shape)

(1500, 200, 200, 3)


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

input_shape = (xsize,ysize, 3)

batch_size = 32
num_classes = 2
epochs = 15

In [0]:
print(x_test.shape)
print(y_test.shape)

(150, 200, 200, 3)
(150,)


In [0]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

model = Sequential()
model.add(Conv2D(32, (2, 2), padding='same', 
                 input_shape=(200, 200, 3), strides=2, activation='relu')) # stride가 이미지 크기와 맞아떨어지지 않을 경우 맞는 부분만 계산(ex> (5, 5) => (2, 2))
model.add(Conv2D(128, (2, 2), activation='relu')) # Kernel의 크기는 input의 크기보다 작아야 한다!(같으면 안됨)
model.add(MaxPooling2D(2, 2))# MaxPooling은 최댓값이 겹치지 않도록(중요) stride를 알아서 조절해 데이터를 축소한다!(통상적으로 절반 ex> (6, 6) => (3, 3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 100, 100, 32)      416       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 99, 99, 128)       16512     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 128)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 307328)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               39338112  
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 387       
Total params: 39,355,427
Trainable params: 39,355,427
Non-trainable params: 0
__________________________________________

In [0]:
model.fit(x_train, y_train, validation_split=0.2, 
          epochs=10, batch_size=10, verbose=1)

acc = model.evaluate(x_test, y_test)
print(acc)

Train on 1080 samples, validate on 270 samples
Epoch 1/10
1080/1080 [==============================] - 5s 4ms/step - loss: 2.4522e-04 - acc: 1.0000 - val_loss: 0.1643 - val_acc: 0.9556
Epoch 2/10
1080/1080 [==============================] - 5s 4ms/step - loss: 1.5543e-04 - acc: 1.0000 - val_loss: 0.1678 - val_acc: 0.9519
Epoch 3/10
1080/1080 [==============================] - 5s 4ms/step - loss: 1.0764e-04 - acc: 1.0000 - val_loss: 0.1581 - val_acc: 0.9519
Epoch 4/10
1080/1080 [==============================] - 5s 4ms/step - loss: 7.2885e-05 - acc: 1.0000 - val_loss: 0.1762 - val_acc: 0.9556
Epoch 5/10
1080/1080 [==============================] - 5s 4ms/step - loss: 5.1504e-05 - acc: 1.0000 - val_loss: 0.1667 - val_acc: 0.9519
Epoch 6/10
1080/1080 [==============================] - 5s 4ms/step - loss: 3.7274e-05 - acc: 1.0000 - val_loss: 0.1731 - val_acc: 0.9519
Epoch 7/10
1080/1080 [==============================] - 5s 4ms/step - loss: 2.6572e-05 - acc: 1.0000 - val_loss: 0.1644 - val

In [0]:
test_path = "/content/drive/My Drive/Colab Notebooks/test_set" # 기쁨만 모인 test set
test_path2 = "/content/drive/My Drive/Colab Notebooks/test_set2" # 무표정만 모인 test set 

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
          pass

In [0]:
# test set 기쁨
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

In [0]:
# test set(무표정)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

In [0]:
# 예측(test set 기쁨)
co=0
for i in range(100):
  print(np.argmax(pred[i]),end=' ')
  tmp=np.argmax(pred[i])

  if tmp == 1:
    co+=1

print('\n',co/100.)

In [0]:
# 예측(test set 무표정)
co=0
for i in range(100):
  print(np.argmax(pred[i]),end=' ')
  tmp=np.argmax(pred[i])

  if tmp == 0:
    co+=1

print('\n',co/100.)